In [ ]:
!pip install transformers torch

In [ ]:
import re
from transformers import BertTokenizer, BertModel
import torch
from scipy.spatial.distance import cosine
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
!pip install transformers torch pandas

In [ ]:
import pandas as pd
import pickle
import numpy as np
from scipy.spatial.distance import cosine
from transformers import BertTokenizer, BertModel
import torch
import os

# Load the given text and generate embedding
def get_embedding_for_text(text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Calculate cosine similarity
def calculate_similarity(embedding1, embedding2):
    embedding1_flat = embedding1.flatten()
    embedding2_flat = embedding2.flatten()
    dot_product = np.dot(embedding1_flat, embedding2_flat)
    norm_vec1 = np.linalg.norm(embedding1_flat)
    norm_vec2 = np.linalg.norm(embedding2_flat)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

# Your given text
given_text = "get maximum number from integer array"
given_text_embedding = get_embedding_for_text(given_text)

# Directory containing your .pkl files
embedding_dir = '/content/drive/MyDrive/Method/EnglishMethod/Score-Examples/PromptExamples/Embedded_English'

# Calculate similarity scores
similarity_scores = []
for file_name in os.listdir(embedding_dir):
    if file_name.endswith('.pkl'):
        row_number = int(file_name.split('_')[1].split('.')[0])
        with open(os.path.join(embedding_dir, file_name), 'rb') as file:
            embedding = pickle.load(file)
        score = calculate_similarity(given_text_embedding, embedding)
        similarity_scores.append((row_number, score))

# Sort by similarity score and get top 30
top_matches = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:30]

# Load the CSV file
csv_path = '/content/drive/MyDrive/Method/EnglishMethod/Score-Examples/PromptExamples/Embedded_English/BestExamples.csv'
df = pd.read_csv(csv_path)

# Filter the DataFrame for the top matches
top_df = df[df.index.isin([num-1 for num, _ in top_matches])]  # Adjusting index if necessary

# Print or save the top 30 functional descriptions and method names
print(top_df[['Method Name', 'Functional Description']])


In [ ]:
# Initialize an empty list to hold each example text
examples_list = []

# Loop through each row in the DataFrame
for index, row in top_df.iterrows():
    # Format the string for the current row and append it to the list
    example = f"Functional Description: \"{row['Functional Description']}\" -> Method Name:\"{row['Method Name']}\""
    examples_list.append(example)

# Join the list of examples into a single string separated by newlines
examples_text = "\n".join(examples_list)



In [ ]:
print(examples_text)

In [ ]:
prompt = f"""Prompt: Suggest a method name according to the given functional description.
Functional Description: {given_text}
Output Requirements:
i. Use Verb-Noun Pairs: Aim to use verb-noun pairs to indicate actions performed on objects, like "calculateTotal" or "findIndex."
ii. Employ CamelCase: Begin with a lowercase letter and capitalize the first letter of each subsequent word.
iii. Specificity Matters: Be as specific as possible without making the method name overly lengthy.
iv. Avoid Java Reserved Words: Ensure the method name does not include Java reserved words or keywords.
v. No Special Characters or Spaces: Use only alphanumeric characters.
Examples:
{examples_text}
"""
print(prompt)


Prompt: Suggest a method name according to the given functional description.
Functional Description: get maximum number from integer array
Output Requirements:
i. Use Verb-Noun Pairs: Aim to use verb-noun pairs to indicate actions performed on objects, like "calculateTotal" or "findIndex."
ii. Employ CamelCase: Begin with a lowercase letter and capitalize the first letter of each subsequent word.
iii. Specificity Matters: Be as specific as possible without making the method name overly lengthy.
iv. Avoid Java Reserved Words: Ensure the method name does not include Java reserved words or keywords.
v. No Special Characters or Spaces: Use only alphanumeric characters.
Examples:
Functional Description: "writes b length bytes from the specified byte array to this file output stream   " -> Method Name:"write"
Functional Description: "returns the java type info for the indexed property   " -> Method Name:"getIndexedPropertyType"
Functional Description: "writes len bytes from the specified byt